# Example Workflow 
## Open Space Benefit vs voting results
Here we want to explore for any relationships between open space benefit score and precinct voting results. 

## Data
* NC Voting districts: https://www.nconemap.gov/datasets/771ae6473a1c4ba3bc768cc2c4b10015_0/explore
* NC Voting Data - https://dl.ncsbe.gov/index.html?prefix=ENRS/2024_11_05/


In [2]:
#Packages
import pandas as pd

from arcgis import GIS
from arcgis.features import FeatureLayer, GeoAccessor

#Connect to AGOL (Anonmously)
gis = GIS()

In [21]:
#Read the NC voting data (downloaded into the data folder)
df = pd.read_csv('./data/results_pct_20241105.txt',sep='\t')

#Wrangle
df = (df
      #Filter for the presidential race
      .loc[(df['Contest Name'] == 'US PRESIDENT')]
      #Group by County, Precinct, and Party - calculating total votes
      .groupby(['County','Precinct','Choice Party'])
      .agg({'Total Votes':'sum'})
      #Pivot so each party has its column
      .reset_index()
      .pivot_table(values='Total Votes',columns='Choice Party',index=['County','Precinct'])
      .reset_index()
     )

#Compute percentages
parties = list(df.columns[2:])
sum_vote = df.loc[:,parties].sum(axis=1)
for p in parties:
    df[f'{p}_pct'] = (df[p] / sum_vote * 100).round(2)
    
#Display
df.head()

Choice Party,County,Precinct,CST,DEM,GRE,JFA,LIB,REP,CST_pct,DEM_pct,GRE_pct,JFA_pct,LIB_pct,REP_pct
0,ALAMANCE,01,1,587,7,2,6,2431,0.03,19.35,0.23,0.07,0.20,80.13
1,ALAMANCE,02,2,632,9,0,17,2558,0.06,19.64,0.28,0.00,0.53,79.49
2,ALAMANCE,035,2,1642,10,8,9,1980,0.05,44.97,0.27,0.22,0.25,54.23
3,ALAMANCE,03C,2,872,21,7,9,1124,0.10,42.85,1.03,0.34,0.44,55.23
4,ALAMANCE,03N,3,1433,7,3,21,1150,0.11,54.76,0.27,0.11,0.80,43.94


In [22]:
#Fetch the precinct features and convert to spatial dataframe
lyr_precincts = FeatureLayer('https://services.nconemap.gov/secure/rest/services/NC1Map_Boundaries/MapServer/0')
sdf_precincts = GeoAccessor.from_layer(lyr_precincts)
sdf_precincts.head()

,objectid,id,prec_id,enr_desc,county_nam,of_prec_id,county_id,shape_leng,st_area(shape),st_perimeter(shape),SHAPE
0,1,1,0003,ALBEMARLE NUMBER 3,STANLY,,84,40731.350779,43231281.474898,40731.350779,"{""rings"": [[[501353.6771267742, 178235.4578594..."
1,2,2,0003,DREXEL 03,BURKE,,12,49918.849039,108319906.647547,49918.84904,"{""rings"": [[[372074.70772677363, 221553.271259..."
2,3,22,0019,LINVILLE 01,BURKE,,12,160866.952809,871805502.088099,160866.952809,"{""rings"": [[[348092.06152677216, 224114.752159..."
3,4,23,0019,RIDENHOUR,STANLY,,84,125698.778851,899272261.409063,125698.778851,"{""rings"": [[[484800.94602677185, 182206.711059..."
4,5,71,007,007,MECKLENBURG,,60,17915.436972,18523838.740518,17915.436972,"{""rings"": [[[446820.3248267737, 159981.3444594..."


In [31]:
#Join voting data
sdf_precincts = lyr_precincts.query("county_nam IN ('WAKE','DURHAM','ORANGE')").sdf
sdf_precincts.head(2)

,objectid,id,prec_id,enr_desc,county_nam,of_prec_id,county_id,shape_leng,st_area(shape),st_perimeter(shape),SHAPE
0,26,495,07,SCHOOL OF THE ARTS,DURHAM,,32,23238.895291,18598541.386539,23238.895291,"{""rings"": [[[617530.541726774, 249263.70725940..."
1,39,105,01-05,01-05,WAKE,,92,14500.10594,13259014.500534,14500.10594,"{""rings"": [[[641594.0037267727, 227539.2814594..."


In [23]:
sdf_precincts_join = pd.merge(
    left=sdf_precincts,
    left_on = ['county_nam','prec_id'],
    right=df,
    right_on= ['County','Precinct'],
    how='left'
)
sdf_precincts_join.head()

,objectid,id,prec_id,enr_desc,county_nam,of_prec_id,county_id,shape_leng,st_area(shape),st_perimeter(shape),...,GRE,JFA,LIB,REP,CST_pct,DEM_pct,GRE_pct,JFA_pct,LIB_pct,REP_pct
0,1,1,0003,ALBEMARLE NUMBER 3,STANLY,,84,40731.350779,43231281.474898,40731.350779,...,4,1,4,175,0.29,46.22,1.16,0.29,1.16,50.87
1,2,2,0003,DREXEL 03,BURKE,,12,49918.849039,108319906.647547,49918.84904,...,2,0,1,736,0.00,20.54,0.22,0.00,0.11,79.14
2,3,22,0019,LINVILLE 01,BURKE,,12,160866.952809,871805502.088099,160866.952809,...,3,0,2,363,0.00,31.73,0.56,0.00,0.37,67.35
3,4,23,0019,RIDENHOUR,STANLY,,84,125698.778851,899272261.409063,125698.778851,...,0,2,1,292,0.00,12.20,0.00,0.60,0.30,86.90
4,5,71,007,007,MECKLENBURG,,60,17915.436972,18523838.740518,17915.436972,...,9,3,11,442,0.07,68.43,0.61,0.20,0.75,29.95


https://developers.arcgis.com/python/latest/guide/visualizing-data-with-the-spatially-enabled-dataframe/  
https://developers.arcgis.com/python-2-3/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.plot

In [25]:
map1 = gis.map('Durham County, NC')

sdf_precincts_join.spatial.plot(
    map_widget=map1,
    palette='viridis',
    renderer_type='c',
    col='DEM_pct',
    alpha=0.1,
    #Class break render options
    method='esriClassifyEqualInterval',
    class_count=10,
    line_width=0.1
)



MapView(layout=Layout(height='400px', width='100%'))